In [1]:
import pandas as pd
import phoenix as px
import numpy as np
import ast

/home/vishesh/anaconda3/envs/alibidetect/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-10 13:52:59.488692: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 13:52:59.560110: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-10 13:53:00.016850: W tensorflow/compiler/xla/stream_executor/

In [2]:
perturbation = "shot_noise"
severity = 10
train_df = pd.read_csv("/home/vishesh/Desktop/datasets/ships-data/train.csv")
# clean_val_df = pd.read_csv("/home/vishesh/Desktop/datasets/ships-data/clean_val.csv")
perturbed_df = pd.read_csv(f"/home/vishesh/Desktop/datasets/ships-data/{perturbation}_{severity}_val.csv")

train_df.drop(columns=['Unnamed: 0'], inplace=True)
# clean_val_df.drop(columns=['Unnamed: 0'], inplace=True)
perturbed_df.drop(columns=['Unnamed: 0'], inplace=True)

train_df['actual'] = train_df['actual'].astype(int)
# clean_val_df['actual'] = clean_val_df['actual'].astype(int)
perturbed_df['actual'] = perturbed_df['actual'].astype(int)

def convert_to_list(string):
    return np.array(ast.literal_eval(string))

train_df['embeddings'] = train_df['embeddings'].apply(convert_to_list)
# clean_val_df['embeddings'] = clean_val_df['embeddings'].apply(convert_to_list)
perturbed_df['embeddings'] = perturbed_df['embeddings'].apply(convert_to_list)

combined_df = perturbed_df
# combined_df = pd.concat([clean_val_df[:900], perturbed_df[-100:]], ignore_index=True)

In [3]:
train_df.shape, combined_df.shape

((14713, 4), (1000, 4))

Refer to this link to launch a web server https://docs.arize.com/phoenix/how-to/define-your-schema#local-images

In [10]:
train_schema = px.Schema(
    prediction_label_column_name="predicted",
    actual_label_column_name="actual",
    embedding_feature_column_names={
        "image_embedding": px.EmbeddingColumnNames(
            vector_column_name="embeddings",
            link_to_data_column_name="url",
        ),
    },
)

In [11]:
prod_schema = px.Schema(
    prediction_label_column_name="predicted",
    actual_label_column_name="actual",
    embedding_feature_column_names={
        "image_embedding": px.EmbeddingColumnNames(
            vector_column_name="embeddings",
            link_to_data_column_name="url",
        ),
    },
)

In [12]:
prod_ds = px.Dataset(dataframe=combined_df, schema=prod_schema, name="production")
train_ds = px.Dataset(dataframe=train_df, schema=train_schema, name="training")

In [13]:
session = px.launch_app(primary=prod_ds, reference=train_ds)

🌍 To view the Phoenix app in your browser, visit http://localhost:32883/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
